In [1]:
import pandas as pd  
import numpy as np
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import apriori

In [2]:
orders = pd.read_csv('data/orders.csv' )
order_products_train = pd.read_csv('data/order_products__train.csv')
order_products_prior = pd.read_csv('data/order_products__prior.csv')
products = pd.read_csv('data/products.csv')
aisles = pd.read_csv('data/aisles.csv')
departments = pd.read_csv('data/departments.csv')

In [20]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [3]:
# Concat order_products_prior and train together to get a total list of ordered products
order_products_total = pd.concat([order_products_prior, order_products_train])

print('there are', order_products_total.shape[0], 'number of products have been ordered')

there are 33819106 number of products have been ordered


In [4]:
# Merge order_products_total with products to get product names
order_products_total = order_products_total.drop('add_to_cart_order', axis = 1)
order_products_total = order_products_total.merge(products[['product_id', 'product_name']],how='left', on='product_id')
order_products_total

,order_id,product_id,reordered,product_name
0,2,33120,1,Organic Egg Whites
1,2,28985,1,Michigan Organic Kale
2,2,9327,0,Garlic Powder
3,2,45918,1,Coconut Butter
4,2,30035,0,Natural Sweetener
...,...,...,...,...
33819101,3421063,14233,1,Natural Artesian Water
33819102,3421063,35548,1,Twice Baked Potatoes
33819103,3421070,35951,1,Organic Unsweetened Almond Milk
33819104,3421070,16953,1,Creamy Peanut Butter


In [5]:
order_products_total['quantity'] = np.zeros(len(order_products_total))+1
idx = order_products_total['reordered'] == 1
order_products_total = order_products_total[idx]
order_products_total = order_products_total[0:500000]

In [7]:
order_products_total['product_id'].nunique()

29540

In [14]:
basket = order_products_total.groupby(['order_id', 'product_id'])['quantity'].sum()
basket = basket.unstack().reset_index().fillna(0).set_index('order_id')
basket

product_id,1,3,4,9,10,12,15,18,23,25,...,49676,49677,49678,49679,49680,49681,49683,49685,49686,49688
order_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
basket[basket > 1] = 1
basket

product_id,1,3,4,9,10,12,15,18,23,25,...,49676,49677,49678,49679,49680,49681,49683,49685,49686,49688
order_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
frequent_itemsets = apriori(basket, min_support = 0.01, use_colnames = True)
frequent_itemsets.sort_values('support', ascending = False)

,support,itemsets
31,0.142155,(24852)
15,0.112777,(13176)
21,0.072495,(21137)
23,0.066435,(21903)
69,0.059782,(47209)
...,...,...
81,0.010297,"(47209, 21137)"
79,0.010229,"(24852, 16797)"
83,0.010175,"(47626, 24852)"
11,0.010094,(9839)


In [19]:
a_rules = association_rules(frequent_itemsets, metric = "lift",
  min_threshold = 0.1)
a_rules.sort_values('lift',ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(13176),(27966),0.112777,0.036652,0.010904,0.096685,2.637917,0.006770,1.066459
5,(27966),(13176),0.036652,0.112777,0.010904,0.297496,2.637917,0.006770,1.262944
6,(13176),(47209),0.112777,0.059782,0.017570,0.155798,2.606074,0.010828,1.113735
7,(47209),(13176),0.059782,0.112777,0.017570,0.293905,2.606074,0.010828,1.256521
12,(47209),(21137),0.059782,0.072495,0.010297,0.172235,2.375827,0.005963,1.120493
13,(21137),(47209),0.072495,0.059782,0.010297,0.142033,2.375827,0.005963,1.095866
19,(47766),(24852),0.047178,0.142155,0.014116,0.299199,2.104732,0.007409,1.224092
18,(24852),(47766),0.142155,0.047178,0.014116,0.099298,2.104732,0.007409,1.057865
8,(24852),(16797),0.142155,0.034965,0.010229,0.071957,2.057967,0.005259,1.039860
9,(16797),(24852),0.034965,0.142155,0.010229,0.292551,2.057967,0.005259,1.212589


In [24]:
print (products.iloc[13175])
print (products.iloc[27965])

product_id                        13176
product_name     Bag of Organic Bananas
aisle_id                             24
department_id                         4
Name: 13175, dtype: object
product_id                     27966
product_name     Organic Raspberries
aisle_id                         123
department_id                      4
Name: 27965, dtype: object


In [28]:
print (products.iloc[24852-1])
print (products.iloc[47626-1])

product_id        24852
product_name     Banana
aisle_id             24
department_id         4
Name: 24851, dtype: object
product_id             47626
product_name     Large Lemon
aisle_id                  24
department_id              4
Name: 47625, dtype: object
